In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from tensorflow.python.keras.utils.np_utils import to_categorical

size_img = 28
threshold_color = 100 / 255


def int2float_grey(x):
    x = x / 255
    return x


def build_dataset():
    # Any results you write to the current directory are saved as output.
    with open("./Dataset/mnist_train.csv") as file:
        data_train = pd.read_csv(file)

        # y_train has the label, x_train has the image data
        y_train = np.array(data_train.iloc[:, 0])
        x_train = np.array(data_train.iloc[:, 1:])

        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                          train_size=0.8, test_size=0.2,
                                                          random_state=1)

    with open("./Dataset/mnist_test.csv") as file:
        data_test = pd.read_csv(file)

        # y_test has the label, x_test has the image data
        y_test = np.array(data_test.iloc[:, 0])
        x_test = np.array(data_test.iloc[:, 1:])

        n_features_train = x_train.shape[1]
        n_samples_train = x_train.shape[0]
        n_samples_test = x_test.shape[0]

        x_train = int2float_grey(x_train)
        x_test = int2float_grey(x_test)

        x_train = np.reshape(x_train, (-1, 28, 28, 1))
        x_test = np.reshape(x_test, (-1, 28, 28, 1))
        x_val = np.reshape(x_val, (-1, 28, 28, 1))

        y_train = to_categorical(y_train)
        y_test = to_categorical(y_test)
        y_val = to_categorical(y_val)

        print(f"Features : {n_features_train}")
        print(f"Number of training samples : {n_samples_train}")
        print(f"Number of Testing Samples : {n_samples_test}")
        print(f"x_train shape : {x_train.shape}")
        print(f"y_train shape : {y_train.shape}")
        print(f"x_test shape : {x_test.shape}")
        print(f"y_test shape : {y_test.shape}")

    return x_train, y_train, x_test, y_test, x_val, y_val

def colab_fetch_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                          train_size=0.8, test_size=0.2,
                                                          random_state=1)
    
    n_features_train = x_train.shape[1]
    n_samples_train = x_train.shape[0]
    n_samples_test = x_test.shape[0]

    x_train = int2float_grey(x_train)
    x_test = int2float_grey(x_test)

    x_train = np.reshape(x_train, (-1, 28, 28, 1))
    x_test = np.reshape(x_test, (-1, 28, 28, 1))
    x_val = np.reshape(x_val, (-1, 28, 28, 1))

    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)

    print(f"Features : {n_features_train}")
    print(f"Number of training samples : {n_samples_train}")
    print(f"Number of Testing Samples : {n_samples_test}")
    print(f"x_train shape : {x_train.shape}")
    print(f"y_train shape : {y_train.shape}")
    print(f"x_test shape : {x_test.shape}")
    print(f"y_test shape : {y_test.shape}")

    return x_train, y_train, x_test, y_test, x_val, y_val


In [ ]:
# Import Keras and Tensorflow
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.python.keras.layers import Conv2D, Dropout, Flatten, Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

# Import Training, Testing and Validation
x_train, y_train, x_test, y_test, x_val, y_val = colab_fetch_mnist()


# Define different model architectures
def model1():
    model = Sequential()
    model.add(Conv2D(20, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(28, 28, 1)))
    # model.add(BatchNormalization())
    model.add(Conv2D(20, kernel_size=(3, 3), activation='relu', strides=2))
    model.add(Dropout(0.2))
    model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
    # model.add(BatchNormalization())
    # model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    return model

def cnn_model():
    model = Sequential()

    # two convolution layers, with one max-pooling and dropout
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # two convolution layers, with one max-pooling and dropout
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # two convolution layers, with one max-pooling and dropout
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # final fully connected layers, for which flattening is needed initially
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    return model


def compile_and_train_model(model, number):
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=["accuracy"])
    datagen = ImageDataGenerator(zoom_range=0.1,
                                 height_shift_range=0.1,
                                 width_shift_range=0.1,
                                 rotation_range=10)

    hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                               steps_per_epoch=500,
                               epochs=30,
                               verbose=2,
                               validation_data=(x_val, y_val),
                               callbacks=[annealer,
                                          ModelCheckpoint(f'model{number}.h5', save_best_only=True)])

    final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
    print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

    plt.plot(hist.history['loss'], color='b')
    plt.plot(hist.history['val_loss'], color='r')
    plt.show()
    plt.plot(hist.history['accuracy'], color='b')
    plt.plot(hist.history['val_accuracy'], color='r')
    plt.show()

    


model = cnn_model()
compile_and_train_model(model, 1)


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('model1.h5')

x_train, y_train, x_test, y_test, x_val, y_val = colab_fetch_mnist()

print("Making Predictions")

# y_pred = model.predict_classes(x_test)
y_pred = np.argmax(model.predict(x_test), axis=-1)
acc = np.sum(y_pred == y_test) / np.size(y_pred)
print("[RESULT] TEST ACCURACY = {}".format(acc))
